In [25]:
import zipfile
import pandas as pd
import nltk

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
archive = zipfile.ZipFile('training_data.zip', 'r')
df = pd.read_csv(archive.open('training.1600000.processed.noemoticon.csv'), header=None, encoding_errors="replace")
# df = pd.read_csv("training_data/training.1600000.processed.noemoticon.csv", encoding="utf-8", encoding_errors="replace")

In [3]:
# target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

# ids: The id of the tweet ( 2087)

# date: the date of the tweet (Sat May 16 23:58:44 UTC 2009)

# flag: The query (lyx). If there is no query, then this value is NO_QUERY.

# user: the user that tweeted (robotickilldozr)

# text: the text of the tweet (Lyx is cool)

df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
data = df.drop([1,2,3,4], axis=1)
# data.columns = ["label","text"]
data.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [19]:
# data = data["text"].dropna()
data[5] = [entry.lower() for entry in data[5]]
data.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - awww, t..."
1,0,is upset that he can't update his facebook by ...
2,0,@kenichan i dived many times for the ball. man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [26]:
data[5]= [word_tokenize(entry) for entry in data[5]]
data.head()

KeyboardInterrupt: 